In [3]:
import pandas as pd
import json
from kafka import KafkaConsumer
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

# Replace this with the address of your Kafka server
kafka_server = "kafka:9092"

# Replace this with the name of the topic you want to consume from
topic_name = "tweets"

# Create a Kafka consumer
consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=kafka_server,
    auto_offset_reset='earliest',
    group_id='influx',
    consumer_timeout_ms=10000
)

bucket = "DSI_project"
org = "dsi"
token = "SKg6fyRg1uVTpil9kbJfEYD4sWLuXET6hUu3Ql6YdLufUZpVbh3GtwmRAqecWWk1o18MvdBoLZ-tKmgoqok6KA=="
# Store the URL of your InfluxDB instance
url="http://influxdb:8086"

client = influxdb_client.InfluxDBClient(
   url=url,
   token=token,
   org=org
)

write_api = client.write_api(write_options=SYNCHRONOUS)

# Read all messages from the topic
for message in consumer:
    entry = json.loads(message.value)
    tweets = int(entry['tweet_count'])
    #convert to date from linux timestamp
    date = pd.to_datetime(entry['time'], unit='ms')
    p1 = influxdb_client.Point("twitter").tag("hashtag", "tsla").field('tweets', tweets)
    p1.time(date)
    write_api.write(bucket=bucket, org=org, record=p1)


# Close the consumer
consumer.close()